In [3]:
!pip install pdfplumber
import pdfplumber
import pandas as pd
import os

class PDFTableExtractor:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.tables = []
        self.page_width = 0
        self.page_height = 0

    def parse_pdf(self):
        with pdfplumber.open(self.pdf_path) as pdf:
            for page in pdf.pages:
                self.page_width = page.width
                self.page_height = page.height
                self._process_page(page)

    def _process_page(self, page):
        words = page.extract_words(keep_blank_chars=True)
        if not words:
            return

     
        rows = self._cluster_rows(words)
        
        table = self._detect_columns(rows)
        if table:
            self.tables.append(table)

    def _cluster_rows(self, words, vertical_tolerance=8):
        rows = []
        current_row = []
        current_y = words[0]['top']

        for word in words:
            if abs(word['top'] - current_y) <= vertical_tolerance:
                current_row.append(word)
            else:
                rows.append(sorted(current_row, key=lambda x: x['x0']))
                current_row = [word]
                current_y = word['top']
        if current_row:
            rows.append(sorted(current_row, key=lambda x: x['x0']))
        return rows

    def _detect_columns(self, rows, horizontal_tolerance=50):
        if not rows:
            return None

        # Find all x0 positions to determine column boundaries
        x_positions = []
        for row in rows:
            x_positions.extend([word['x0'] for word in row])
        
        # Cluster x positions into columns
        sorted_x = sorted(x_positions)
        clusters = []
        current_cluster = [sorted_x[0]]
        
        for x in sorted_x[1:]:
            if x - current_cluster[-1] <= horizontal_tolerance:
                current_cluster.append(x)
            else:
                clusters.append(current_cluster)
                current_cluster = [x]
        clusters.append(current_cluster)
        
        
        column_x = [sum(cluster)/len(cluster) for cluster in clusters]
        column_x = sorted(column_x)
        
   
        table = []
        for row in rows:
            table_row = []
            current_col = 0
            for word in row:
                
                while current_col < len(column_x) and word['x0'] > column_x[current_col] + horizontal_tolerance:
                    table_row.append('')  
                    current_col += 1
                if current_col >= len(column_x):
                    break
                table_row.append(word['text'])
                current_col += 1
            # Fill remaining columns
            while len(table_row) < len(column_x):
                table_row.append('')
            table.append(table_row)
        
        return table

    def to_excel(self, output_path):
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            for i, table in enumerate(self.tables):
                df = pd.DataFrame(table)
                df.to_excel(writer, sheet_name=f'Table_{i+1}', index=False, header=False)

def process_pdf(input_path, output_folder):
    extractor = PDFTableExtractor(input_path)
    extractor.parse_pdf()
    output_path = os.path.join(output_folder, f"{os.path.basename(input_path)}.xlsx")
    extractor.to_excel(output_path)
    print(f"Saved {output_path}")


if __name__ == "__main__":
    input_pdf = r"C:\Users\sushant kaddu\Downloads\test3 (1).pdf"  
    output_path = r"C:\Users\sushant kaddu\Desktop\machine learning projects" 
 
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    process_pdf(input_pdf, output_path)

Saved C:\Users\sushant kaddu\Desktop\machine learning projects\test3 (1).pdf.xlsx


In [13]:
pip install pandas openpyxl

In [33]:
import pandas as pd
import re
from datetime import datetime
from openpyxl.utils import get_column_letter

def parse_transaction(cell_content):
    """Improved parser for transaction cells"""
    parts = re.split(r'\s{2,}', cell_content)  
    transaction = {
        'Date': '',
        'Type': '',
        'Description': '',
        'Transaction Amount': '',
        'Running Balance': ''
    }

    try:
        
        date_type = parts[0].split()
        transaction['Date'] = datetime.strptime(date_type[0], '%d-%b-%Y').strftime('%d-%b-%Y')
        transaction['Type'] = date_type[1] if len(date_type) > 1 else ''
        
        
        amount_balance = [p for p in parts[1:] if re.match(r'^[\d,.]+[DC]?r?$', p)]
        if len(amount_balance) >= 2:
            transaction['Transaction Amount'] = amount_balance[-2].replace(',', '')
            transaction['Running Balance'] = amount_balance[-1].replace(',', '')
        
     
        desc_parts = parts[1:-2] if len(amount_balance) >= 2 else parts[1:]
        transaction['Description'] = ' '.join(desc_parts).strip()

    except Exception as e:
        print(f"Error parsing: {cell_content}\n{str(e)}")
    
    return transaction

def process_excel(input_path, output_path):
    all_sheets = pd.read_excel(input_path, sheet_name=None, header=None)
    transactions = []
    
    for sheet_name, df in all_sheets.items():
        current_transaction = None
        
        for _, row in df.iterrows():
            cell_content = str(row[0]).strip() if pd.notna(row[0]) else ''
            
            if re.match(r'\d{2}-[A-Za-z]{3}-\d{4}', cell_content):
                if current_transaction:
                    transactions.append(current_transaction)
                current_transaction = parse_transaction(cell_content)
            elif current_transaction:
                
                current_transaction['Description'] += ' ' + cell_content
                
                
                amounts = re.findall(r'[\d,]+\.\d{2}', cell_content)
                if len(amounts) >= 2:
                    current_transaction['Transaction Amount'] = amounts[-2].replace(',', '')
                    current_transaction['Running Balance'] = amounts[-1].replace(',', '')
    
    if current_transaction:
        transactions.append(current_transaction)

    
    transactions_df = pd.DataFrame(transactions)
    transactions_df = transactions_df[['Date', 'Type', 'Description', 
                                         'Transaction Amount', 'Running Balance']]
    
    
    numeric_cols = ['Transaction Amount', 'Running Balance']
    for col in numeric_cols:
        transactions_df[col] = pd.to_numeric(transactions_df[col], errors='coerce')
    
    
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        transactions_df.to_excel(writer, index=False, sheet_name='Transactions')
        
       
        worksheet = writer.sheets['Transactions']
        for idx, col in enumerate(transactions_df.columns):
            max_len = max(
                transactions_df[col].astype(str).map(len).max(),
                len(col)
            )  # <-- Closing parenthesis added here
            worksheet.column_dimensions[get_column_letter(idx+1)].width = max_len + 2


input_excel = r"C:\Users\sushant kaddu\Desktop\machine learning projects\test3 (1).pdf.xlsx"
output_excel = r"output_correct_second.xlsx"

process_excel(input_excel, output_excel)
print(f"✅ Transactions saved to: {output_excel}")


✅ Transactions saved to: output_correct_second.xlsx
